# LLM Классификация Отзывов: Полный Пайплайн (Промпт + BERT)

Цель: Классифицировать test.csv по категориям из categories.txt.  
Метрика: Weighted F1 (>0.90 цель).  
Ограничения: Только train для обучения/валидации; <5 сек/пример; Colab T4.

Этапы:
1. Установка.
2. Загрузка данных.
3. Разметка train (Mistral zero-shot).
4. Аугментация few-shot.
5. Промпт-инжиниринг (few-shot CoT).
6. Fine-tune BERT (Google multilingual) — опционально для +F1.
7. Инференс на test + submission.csv.

In [ ]:
!pip install transformers accelerate bitsandbytes datasets pandas scikit-learn

In [ ]:
!pip install torch torchvision

In [ ]:
!pip install peft

In [ ]:
import torch
import pandas as pd
import numpy as np
import time
import re
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import evaluate
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    AutoModelForSequenceClassification, TrainingArguments, Trainer,
    DataCollatorWithPadding, pipeline
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
print(f"GPU: {torch.cuda.is_available()}. VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Разметка Train (Mistral Zero-Shot для Few-Shot)
Валидация только на holdout из train (test.csv не трогаем!).

In [ ]:
# Категории
with open('categories.txt', 'r', encoding='utf-8') as f:
    categories = [line.strip() for line in f.readlines()]
print("Категории:", categories)

# Train и Test
train_df = pd.read_csv('train.csv', names=['text'])  # Убираем кавычки и переносы
train_df['text'] = train_df['text'].str.strip().str.replace(r'^"', '', regex=True).str.replace(r'"$', '', regex=True)

test_df = pd.read_csv('test.csv', names=['text'])
test_df['text'] = test_df['text'].str.strip().str.replace(r'^"', '', regex=True).str.replace(r'"$', '', regex=True)

print(f"Train: {len(train_df)} отзывов\nTest: {len(test_df)} отзывов")

In [ ]:
model_name_mistral = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer_mistral = AutoTokenizer.from_pretrained(model_name_mistral)
tokenizer_mistral.pad_token = tokenizer_mistral.eos_token

model_mistral = AutoModelForCausalLM.from_pretrained(
    model_name_mistral,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

def generate_response_mistral(prompt, max_new_tokens=50, temperature=0.1):
    inputs = tokenizer_mistral(prompt, return_tensors="pt", truncation=True, padding=True).to("cuda")
    with torch.no_grad():
        outputs = model_mistral.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer_mistral.eos_token_id,
            eos_token_id=tokenizer_mistral.eos_token_id
        )
    response = tokenizer_mistral.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip()
    return response

Using device: cuda


In [ ]:
def label_review_zero_shot(review_text):
    prompt = f"""Ты эксперт по классификации отзывов на маркетплейсе. У тебя есть категории товаров: {categories_str}.

Правила:
- Выбери ОДНУ категорию, которая лучше всего подходит к отзыву. Если отзыв не упоминает товар или категория не подходит — "нет товара".
- Шаг 1: Выдели ключевые слова/темы в отзыве (одежда: блузка, юбка, ткань, швы, размер; обувь: ботинки, кроссовки; бытовая техника: стиралка, холодильник, пылесос; посуда: тарелки, кастрюли, ножи; текстиль: постельное, полотенца, шарфы; товары для детей: игрушки, одежда детская, коляска; украшения и аксессуары: кольца, сумки, часы; электроника: телефон, наушники, зарядка).
- Шаг 2: Сопоставь с категориями. Если неоднозначно (только жалобы на доставку) — "нет товара".
- Шаг 3: Ответь ТОЛЬКО названием категории (например, "одежда").

Отзыв: {review_text}
Категория:"""
    response = generate_response_mistral(prompt)
    match = re.search(r'(бытовая техника|обувь|одежда|посуда|текстиль|товары для детей|украшения и аксессуары|электроника|нет товара)', response)
    return match.group(0) if match else "нет товара"

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [ ]:
train_labels = []
start_time = time.time()
for i in range(0, len(train_df), 10):
    batch = train_df['text'][i:i+10].tolist()
    batch_labels = [label_review_zero_shot(text) for text in batch]
    train_labels.extend(batch_labels)
    print(f"Разметили {i+len(batch)}/{len(train_df)}")
avg_time_label = (time.time() - start_time) / len(train_df)
print(f"Среднее время разметки train: {avg_time_label:.2f} сек")

train_df['label'] = train_labels
train_df.to_csv('train_labeled.csv', index=False)

holdout = train_df.sample(frac=0.1, random_state=42)
print("Разметка готова")

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

## Аугментация Few-Shot Примеров
Выберем 12+ примеров (баланс) и аугментируем парафразами.

In [ ]:
# Выбор few-shot (2 на категорию)
few_shot_df = train_df.groupby('label').apply(lambda x: x.sample(min(2, len(x)), random_state=42)).reset_index(drop=True)
few_shot_examples = few_shot_df.to_dict('records')[:12]  # Топ-12

# Парафразинг для аугментации
def paraphrase_example(example_text):
    para_prompt = f"Перефразируй этот отзыв, сохраняя смысл и ключевые слова: {example_text}"
    para_text = generate_response_mistral(para_prompt, max_new_tokens=100)
    return para_text

augmented_examples = []
for ex in few_shot_examples:
    augmented_examples.append({'text': ex['text'], 'label': ex['label']})
    for _ in range(2):  # 2 парафразa
        para_text = paraphrase_example(ex['text'])
        augmented_examples.append({'text': para_text, 'label': ex['label']})

few_shot_str = "\n".join([f"Отзыв: {ex['text']}\nКатегория: {ex['label']}\n" for ex in augmented_examples[:12]])
print("Few-shot готов. Примеров:", len(augmented_examples))

## Промпт-Инжиниринг (Few-Shot CoT для Test)

In [ ]:
# Few-shot CoT промпт
def classify_prompt(review_text):
    prompt = f"""Ты — строгий эксперт-аналитик по отзывам маркетплейса Wildberries/Ozon. Твоя задача: точно классифицировать отзыв по категории товара. Доступные категории: {categories_str}. Всегда выбирай ОДНУ.

Правила (соблюдай строго для точности):
1. Проанализируй отзыв шаг за шагом (CoT): 
   - Шаг 1: Выдели ключевые элементы (товар, проблемы: размер/ткань/швы — одежда; посуда/кухня/тарелки — посуда; гаджеты/зарядка/экран — электроника; стиралка/холодильник/механизм — бытовая техника; ботинки/подошва — обувь; постель/шторы/полотенца — текстиль; дети/игрушки/коляска — товары для детей; бижутерия/сумки/часы — украшения и аксессуары).
   - Шаг 2: Сопоставь с категорией. Если товар не упомянут, только доставка/деньги/спор — "нет товара". Если неоднозначно (например, "вещь" без деталей) — "нет товара".
   - Шаг 3: Убедись, что категория логична (не путай одежду с текстилем, если нет постельного).
2. Игнорируй эмоции/доставку, фокусируйся на товаре.
3. Ответь ТОЛЬКО названием категории (без объяснений или номеров).

Примеры:
{few_shot_str}

Новый отзыв: {review_text}
Шаг 1: ...
Шаг 2: ...
Шаг 3: ...
Категория:"""
    response = generate_response_mistral(prompt, max_new_tokens=20, temperature=0.1)
    match = re.search(r'(бытовая техника|обувь|одежда|посуда|текстиль|товары для детей|украшения и аксессуары|электроника|нет товара)', response)
    return match.group(0) if match else "нет товара"


test_prompt_labels = []
start_time = time.time()
for i in range(0, len(test_df), 5):
    batch = test_df['text'][i:i+5].tolist()
    batch_labels = [classify_prompt(text) for text in batch]
    test_prompt_labels.extend(batch_labels)
avg_time_prompt = (time.time() - start_time) / len(test_df)
print(f"Среднее время промпт на test: {avg_time_prompt:.2f} сек")
test_df['predicted_category_prompt'] = test_prompt_labels

## Fine-Tune BERT (Google Multilingual) — Опционально для Топ-F1
Заменит промпт на supervised модель. Валидация только на holdout из train.

In [ ]:
# Подготовка данных для BERT (только train)
le = LabelEncoder()
train_df['label_id'] = le.fit_transform(train_df['label'])
num_labels = len(le.classes_)

train_split, val_split = train_test_split(train_df, test_size=0.1, stratify=train_df['label_id'], random_state=42)
train_dataset = Dataset.from_pandas(train_split[['text', 'label_id']])
val_dataset = Dataset.from_pandas(val_split[['text', 'label_id']])

# Аугментация: Шум в train
def augment_text(text):
    words = text.split()
    if len(words) > 3 and np.random.rand() > 0.5:
        np.random.shuffle(words)
        return ' '.join(words)
    return text

train_dataset = train_dataset.map(lambda x: {'text': augment_text(x['text'])})
model_name_bert = "google-bert/bert-base-multilingual-cased"
tokenizer_bert = AutoTokenizer.from_pretrained(model_name_bert)
if tokenizer_bert.pad_token is None:
    tokenizer_bert.pad_token = tokenizer_bert.unk_token

def tokenize_function(examples):
    return tokenizer_bert(examples['text'], truncation=True, padding=True, max_length=256)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label_id'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label_id'])

In [ ]:
# Модель BERT + LoRA
model_bert = AutoModelForSequenceClassification.from_pretrained(
    model_name_bert,
    num_labels=num_labels,
    id2label={i: label for i, label in enumerate(le.classes_)},
    label2id={label: i for i, label in enumerate(le.classes_)}
)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["query", "value"]
)
model_bert = get_peft_model(model_bert, lora_config)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer_bert)

metric = evaluate.load("f1")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [ ]:
# Training
training_args = TrainingArguments(
    output_dir="./bert-finetuned",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    report_to="none"
)

trainer = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer_bert,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
val_results = trainer.evaluate()
print(f"Валидационный Weighted F1 (holdout из train): {val_results['eval_f1']:.4f}")

trainer.save_model("./bert-final")
tokenizer_bert.save_pretrained("./bert-final")

## Инференс на Test + Submission
'prompt' или 'bert' (по умолчанию BERT, если обучен).

In [ ]:
method = 'bert'  # Или 'prompt'

if method == 'bert':
    classifier = pipeline("text-classification", model="./bert-final", tokenizer="./bert-final", device=0)
    def classify_bert(text):
        result = classifier(text)
        pred_id = int(result[0]['label'].split('_')[-1]) if '_' in result[0]['label'] else int(result[0]['label'])
        return le.classes_[pred_id]

    start_time = time.time()
    test_df['predicted_category'] = test_df['text'].apply(classify_bert)
    avg_time_bert = (time.time() - start_time) / len(test_df)
    print(f"Среднее время BERT на test: {avg_time_bert:.2f} сек")
else:  # Промпт
    test_df['predicted_category'] = test_df['predicted_category_prompt']
    print(f"Используем промпт. Время: {avg_time_prompt:.2f} сек")

# Submission CSV (формат: category\npred1\n...)
test_df[['predicted_category']].to_csv('submission.csv', index=False, header=False)
print("\nsubmission.csv готов! Статистика:")
print(test_df['predicted_category'].value_counts())

# Пример валидации на holdout (если есть manual_labels)
# f1_final = f1_score(manual_labels, holdout['predicted_category'].tolist()[:len(manual_labels)], average='weighted')
# print(f"Примерный F1: {f1_final}")
print("Готово! Скачай submission.csv.")